In [ ]:
# Load TensorBoard extension
%load_ext tensorboard

# Start TensorBoard
from datetime import datetime
import os

# Set up a log directory for TensorBoard
log_dir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(log_dir, exist_ok=True)


# https://github.com/vishal1905/Super-Resolution/blob/master/SR_PT.ipynb
# Has been the source of reference for much of the changes made to convert to a SRGAN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#INPORT DATA AND PREPROCESS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore
import os
import cv2
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import torchvision.utils
from torchvision import datasets, transforms
import numpy as np


In [ ]:
#preprocess images to lower resolution
def preProcessing(OUTPUT_PATH):
    LABELS_PATH = '/content/drive/My Drive/OCTDL_labels.csv'
    INPUT_PATH = '/content/drive/My Drive/OCTDL_AUGMENTED'
    #OUTPUT_PATH = '/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED'

    df = pd.read_csv(LABELS_PATH)

    dimensions = pd.DataFrame(df, columns = ['file_name', 'image_width', 'image_hight'])

    dimensions['z_width'] = zscore(dimensions['image_width'])
    dimensions['z_height'] = zscore(dimensions['image_hight'])

    z_threshold = 2.5

    filtered_dimensions = dimensions[
        (abs(dimensions['z_width']) <= z_threshold) &
        (abs(dimensions['z_height']) <= z_threshold)
    ].copy()


    filtered_dimensions['aspect_ratio'] = filtered_dimensions['image_width'] / filtered_dimensions['image_hight']
    avg_aspect_ratio = filtered_dimensions['aspect_ratio'].mean()

    # We will try to make the SRgan work with 64x64 and scale up to the dimensions
    # of other gans later
    height = 64
    width = 64
    target_size = (width, height)

    def resize_and_save(classname):
        input_dir = INPUT_PATH + '/augmented_data/' + classname
        output_dir = OUTPUT_PATH + '/' + classname

        os.makedirs(output_dir, exist_ok = True)

        for _, row in filtered_dimensions.iterrows():
                file_name = row['file_name'] + '.jpg'
                if classname.lower() not in file_name:
                    continue
                image_path = os.path.join(input_dir, file_name)

                # Read the image
                img = cv2.imread(image_path)
                if img is None:
                    print(f"Error loading image {file_name}")
                    continue

                resized_img = cv2.resize(img, target_size, interpolation = cv2.INTER_AREA)

                output_path = os.path.join(output_dir, file_name)
                cv2.imwrite(output_path, resized_img)

    classnames = ['AMD', 'DME', 'ERM', 'NO', 'RAO', 'RVO', 'VID']

    #for classname in classnames:
    #    resize_and_save(classname)


    # hyperparameters - to be adjusted
    # currently 70/30 split
    TEST_TRAIN_SPLIT = 0.3
    # we'll try 32 for now
    BATCH_SIZE = 32

    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor()
    ])

    num_channels = 1 # used later when declaring generator/descriminator
    features_discrim = 16
    # 64 by 64 images
    dataset = datasets.ImageFolder(OUTPUT_PATH, transform=transform)

    #full preprocessed images
    #dataset = datasets.ImageFolder(INPUT_PATH, transform=transform)

    n_test = int(np.floor(TEST_TRAIN_SPLIT * len(dataset)))
    n_train = len(dataset) - n_test

    train_ds, test_ds = random_split(dataset, [n_train, n_test])

    train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)
    test_dl = DataLoader(test_ds, batch_size = BATCH_SIZE, shuffle = False, num_workers = 4)


    # some useful info about the dataset
    print(f"Classes: {dataset.classes}")
    print(f"Number of training samples: {len(train_ds)}")
    print(f"Number of testing samples: {len(test_ds)}")
    #for i, (x, label) in enumerate(train_dl):
        #print(label)
        #break
        #print(f"Image shape: {x.shape}")
    return train_dl, test_dl

#Declaring Generator

In [ ]:
import torch
import torch.nn as nn


# Using https://github.com/vishal1905/Super-Resolution as a reference framework for the SRGAN
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,64,9,padding=4,bias=False)
        self.conv2 = nn.Conv2d(64,64,3,padding=1,bias=False)
        self.conv3_1 = nn.Conv2d(64,256,3,padding=1,bias=False)
        self.conv3_2 = nn.Conv2d(64,256,3,padding=1,bias=False)
        self.conv4 = nn.Conv2d(64,3,9,padding=4,bias=False)
        self.bn = nn.BatchNorm2d(64)
        self.ps = nn.PixelShuffle(2)
        self.prelu = nn.PReLU()

    def forward(self, input):
        block1 = self.prelu(self.conv1(input))
        block2 = torch.add(self.bn(self.conv2(self.prelu(self.bn(self.conv2(block1))))),block1)
        block3 = torch.add(self.bn(self.conv2(self.prelu(self.bn(self.conv2(block2))))),block2)
        block4 = torch.add(self.bn(self.conv2(self.prelu(self.bn(self.conv2(block3))))),block3)
        block5 = torch.add(self.bn(self.conv2(self.prelu(self.bn(self.conv2(block4))))),block4)
        block6 = torch.add(self.bn(self.conv2(self.prelu(self.bn(self.conv2(block5))))),block5)
        block7 = torch.add(self.bn(self.conv2(block6)),block1)
        block8 = self.prelu(self.ps(self.conv3_1(block7)))
        block9 = self.prelu(self.ps(self.conv3_2(block8)))
        block10 = self.conv4(block9)
        return block10

device = "cuda" if torch.cuda.is_available() else "cpu"
# Create the generator
netG = Generator().to("cuda")

# Handle multi-GPU if desired
#if (device.type == 'cuda') and (ngpu > 1):
#    netG = nn.DataParallel(netG, list(range(ngpu)))


# Print the model
print(netG)

Generator(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), bias=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv3_1): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv3_2): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv4): Conv2d(64, 3, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ps): PixelShuffle(upscale_factor=2)
  (prelu): PReLU(num_parameters=1)
)


#Declaring Discriminator

In [ ]:
# Using https://github.com/vishal1905/Super-Resolution as a reference framework for the SRGAN
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,64,3,padding=1,bias=False)
        self.conv2 = nn.Conv2d(64,64,3,stride=2,padding=1,bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64,128,3,padding=1,bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128,128,3,stride=2,padding=1,bias=False)
        self.bn4 = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128,256,3,padding=1,bias=False)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256,256,3,stride=2,padding=1,bias=False)
        self.bn6 = nn.BatchNorm2d(256)
        self.conv7 = nn.Conv2d(256,512,3,padding=1,bias=False)
        self.bn7 = nn.BatchNorm2d(512)
        self.conv8 = nn.Conv2d(512,512,3,stride=2,padding=1,bias=False)
        self.bn8 = nn.BatchNorm2d(512)
        self.fc1 = nn.Linear(512*16*16,1024)
        self.fc2 = nn.Linear(1024,1)
        self.drop = nn.Dropout2d(0.3)

    def forward(self,input):
        block1 = nn.functional.leaky_relu(self.conv1(input))
        block2 = nn.functional.leaky_relu(self.bn2(self.conv2(block1)))
        block3 = nn.functional.leaky_relu(self.bn3(self.conv3(block2)))
        block4 = nn.functional.leaky_relu(self.bn4(self.conv4(block3)))
        block5 = nn.functional.leaky_relu(self.bn5(self.conv5(block4)))
        block6 = nn.functional.leaky_relu(self.bn6(self.conv6(block5)))
        block7 = nn.functional.leaky_relu(self.bn7(self.conv7(block6)))
        block8 = nn.functional.leaky_relu(self.bn8(self.conv8(block7)))
        block8 = block8.view(-1,block8.size(1)*block8.size(2)*block8.size(3))
        block9 = nn.functional.leaky_relu(self.fc1(block8),)
#         block9 = block9.view(-1,block9.size(1)*block9.size(2)*block9.size(3))
        block10 = torch.sigmoid(self.drop(self.fc2(block9)))
        return block9,block10

device = "cuda" if torch.cuda.is_available() else "cpu"
# Create the Discriminator
netD = Discriminator().to("cuda")

# Handle multi-GPU if desired
#if (device.type == 'cuda') and (ngpu > 1):
#    netD = nn.DataParallel(netD, list(range(ngpu)))

# Print the model
print(netD)

Discriminator(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

#initialise loss function and optimisers

In [ ]:
import torch.optim as optim
# Initialize 4 loss functions. SRGANs use a variety of loss functions.
gen_loss = nn.BCELoss()
vgg_loss = nn.MSELoss()
mse_loss = nn.MSELoss()
disc_loss = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, 100, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

LEARNING_RATE = 0.0002
# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

#Begin Training

In [ ]:
# Training Loop
from tqdm import tqdm
# Using https://github.com/vishal1905/Super-Resolution as a reference for the implementation of loss functions
# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
num_epochs = 50
train_lr, test_lr, = preProcessing('/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED') # 64x64 images
train_hr, test_hr, = preProcessing('/content/drive/My Drive/OCTDL_SRGAN_AUGMENTED') # 256x256 images
print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    d1loss_list=[]
    d2loss_list=[]
    gloss_list=[]
    vloss_list=[]
    mloss_list=[]
    for batch in tqdm(range(1)):
        lr_images = np.stack(train_lr)
        hr_images = np.stack(train_hr)


        netD.zero_grad()

        gen_out = netG(torch.from_numpy(lr_images).to("cuda").float())
        _,f_label = netD(gen_out)
        _,r_label = netD(torch.from_numpy(hr_images).to("cuda").float())
        d1_loss = (disc_loss(f_label,torch.zeros_like(f_label,dtype=torch.float)))
        d2_loss = (disc_loss(r_label,torch.ones_like(r_label,dtype=torch.float)))
        # d_loss = d1_loss+d2_loss
        d2_loss.backward()
        d1_loss.backward(retain_graph=True)
        # print(d1_loss,d2_loss)
#         d_loss.backward(retain_graph=True)
        optimizerD.step()


        netG.zero_grad()
        g_loss = gen_loss(f_label.data,torch.ones_like(f_label,dtype=torch.float))
        m_loss = mse_loss(gen_out,torch.from_numpy(hr_images).to("cuda").float())

        generator_loss = g_loss + m_loss
        # v_loss.backward(retain_graph=True)
        # m_loss.backward(retain_graph=True)
        # g_loss.backward()
        # print(generator_loss)

        generator_loss.backward()
        optimizerG.step()

        d1loss_list.append(d1_loss.item())
        d2loss_list.append(d2_loss.item())

        gloss_list.append(g_loss.item())
        mloss_list.append(m_loss.item())



#         print("d1Loss ::: "+str((d1_loss.item()))+" d2Loss ::: "+str((d2_loss.item())))
#         print("gloss ::: "+str((g_loss.item()))+" vloss ::: "+str((v_loss.item()))+" mloss ::: "+str((m_loss.item())))
    print("Epoch ::::  "+str(epoch+1)+"  d1_loss ::: "+str(np.mean(d1loss_list))+"  d2_loss :::"+str(np.mean(d2loss_list)))
    print("genLoss ::: "+str(np.mean(gloss_list))+"  vggLoss ::: "+str(np.mean(vloss_list))+"  MeanLoss  ::: "+str(np.mean(mloss_list)))

Starting Training Loop...


  0%|          | 0/1 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.20 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.02 GiB is free. Process 2530 has 13.72 GiB memory in use. Of the allocated memory 12.55 GiB is allocated by PyTorch, and 1.05 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print(train_lr)

#Plot Results

save the models

In [ ]:
torch.save(netG.state_dict, '/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/netGSR.pth')
torch.save(netD.state_dict, '/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/netDSR.pth')
torch.save(netG, '/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/netGSRfull.pth')
torch.save(netD, '/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/netDSRfull.pth')
netG.eval()
netD.eval()